<a href="https://colab.research.google.com/github/d61h6k4/EML/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 47.8 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorboard
    Found existing 

In [3]:
import pathlib

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-01-21 16:03:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2023-01-21 16:03:41 (57.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [5]:
text = pathlib.Path("input.txt").read_text()

In [6]:
print(f"Length of the text in characters: {len(text)}")

Length of the text in characters: 1115394


In [7]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [10]:
import tensorflow as tf

In [11]:
data = tf.constant(encode(text), dtype=tf.int64)

In [12]:
print(data.shape, data.dtype)
print(data[:100])

(1115394,) <dtype: 'int64'>
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)


In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [14]:
block_size = 8
train_data[:block_size+1]

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([18, 47, 56, 57, 58,  1, 15, 47, 58])>

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} the target: {target}")


when input is [18] the target: 47
when input is [18 47] the target: 56
when input is [18 47 56] the target: 57
when input is [18 47 56 57] the target: 58
when input is [18 47 56 57 58] the target: 1
when input is [18 47 56 57 58  1] the target: 15
when input is [18 47 56 57 58  1 15] the target: 47
when input is [18 47 56 57 58  1 15 47] the target: 58


In [16]:
def get_dataset(split, batch_size=4, block_size=8):
    gen = tf.random.Generator.from_seed(1337)
    data = train_data if split == "train" else val_data

    maxval = len(data) - block_size

    while True:
        ix = gen.uniform((batch_size,), maxval=len(data) - block_size, dtype=tf.int32)
        x = tf.stack([data[i:i+block_size] for i in ix])
        y = tf.stack([data[i+1:i+block_size+1] for i in ix])
        yield x, y


batch_size=4
block_size=8

xb, yb = next(get_dataset("train", batch_size=batch_size, block_size=block_size))
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b, t]
        print(f"when input is {context.numpy().tolist()} the target: {target}")

inputs:
(4, 8)
tf.Tensor(
[[61 57  6  1 47 52  1 39]
 [56 57 11  0 35 46 47 41]
 [ 0 28 56 53 42 47 45 47]
 [46 43  1 52 53 40 50 43]], shape=(4, 8), dtype=int64)
targets:
(4, 8)
tf.Tensor(
[[57  6  1 47 52  1 39 52]
 [57 11  0 35 46 47 41 46]
 [28 56 53 42 47 45 47 53]
 [43  1 52 53 40 50 43 57]], shape=(4, 8), dtype=int64)
----
when input is [61] the target: 57
when input is [61, 57] the target: 6
when input is [61, 57, 6] the target: 1
when input is [61, 57, 6, 1] the target: 47
when input is [61, 57, 6, 1, 47] the target: 52
when input is [61, 57, 6, 1, 47, 52] the target: 1
when input is [61, 57, 6, 1, 47, 52, 1] the target: 39
when input is [61, 57, 6, 1, 47, 52, 1, 39] the target: 52
when input is [56] the target: 57
when input is [56, 57] the target: 11
when input is [56, 57, 11] the target: 0
when input is [56, 57, 11, 0] the target: 35
when input is [56, 57, 11, 0, 35] the target: 46
when input is [56, 57, 11, 0, 35, 46] the target: 47
when input is [56, 57, 11, 0, 35, 46, 47

In [17]:
from tensorflow import keras

In [18]:
class Head(keras.layers.Layer):
    def __init__(self, block_size, head_size, dropout_rate=0.1):
        super().__init__()
        self.key = keras.layers.Dense(head_size, use_bias=False)
        self.query = keras.layers.Dense(head_size, use_bias=False)
        self.value = keras.layers.Dense(head_size, use_bias=False)

        self.dropout = keras.layers.Dropout(dropout_rate)

    def call(self, x):
        T = tf.shape(x)[1]
        C = tf.shape(x)[2]

        k = self.key(x)
        q = self.query(x)
        wei = k @ tf.transpose(q, perm=[0, 2, 1]) * tf.math.rsqrt(tf.cast(C, tf.float32))
        wei = tf.nn.softmax(tf.where(tf.linalg.band_part(tf.ones((T, T)), -1, 0) == 1, wei, float('-inf')), axis=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out


In [19]:
class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, block_size, heads_num, head_size, dropout_rate=0.1):
        super().__init__()
        self.heads = keras.Sequential([Head(block_size, head_size, dropout_rate=dropout_rate) for _ in range(heads_num)])
        self.proj = keras.layers.Dense(heads_num * head_size)
        self.dropout = keras.layers.Dropout(dropout_rate)

    def call(self, x):
        out = tf.concat(self.heads(x), axis=-1)
        out = self.dropout(self.proj(out))
        return out

In [20]:
class FeedForward(keras.layers.Layer):
    def __init__(self, n_embed, dropout_rate=0.1):
        super().__init__()
        self.net = keras.Sequential([
            keras.layers.Dense(4 * n_embed, activation="gelu", use_bias=False),
            keras.layers.Dense(n_embed, use_bias=False),
            keras.layers.Dropout(dropout_rate)
            ])

    def call(self, x):
        return self.net(x)

In [21]:
class Block(keras.layers.Layer):
    def __init__(self, block_size, n_embed, n_head, dropout_rate=0.1):
        super().__init__()

        head_size = n_embed // n_head
        self.sa_head = keras.layers.MultiHeadAttention(num_heads=n_head, key_dim=head_size, value_dim=None, dropout=dropout_rate, use_bias=False) # MultiHeadAttention(block_size, n_head, head_size, dropout_rate=dropout_rate)
        self.ffwd = FeedForward(n_embed, dropout_rate=dropout_rate)
        self.ln1 = keras.layers.LayerNormalization()
        self.ln2 = keras.layers.LayerNormalization()

    def call(self, x):
        kv = self.ln1(x)
        x = x + self.sa_head(kv, kv, use_causal_mask=True)
        x = x + self.ffwd(self.ln2(x))
        return x


In [22]:


class BigramLanguageModel(keras.Model):
    def __init__(self, vocab_size, block_size, n_layer, n_head, n_embed=32, dropout_rate=0.1):
        super().__init__()
        self.block_size = block_size

        self.token_embedding_table = keras.layers.Embedding(vocab_size, n_embed)
        self.position_embedding_table = keras.layers.Embedding(block_size, n_embed)

        self.blocks = keras.Sequential([
            Block(block_size, n_embed, n_head, dropout_rate=dropout_rate) for _ in range(n_layer)
            
        ])
        self.ls_ln = keras.layers.LayerNormalization()
        self.lm_embed = keras.layers.Dense(vocab_size)

    def compile(self):
        super().compile(optimizer=keras.optimizers.experimental.AdamW(learning_rate=3e-4),
                        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        jit_compile=True)

    def call(self, idx):
        T = tf.shape(idx)[1]

        token_embed = self.token_embedding_table(idx)
        pos_embed = self.position_embedding_table(tf.range(T))
        x = token_embed + pos_embed
        x = self.blocks(x)
        x = self.ls_ln(x)
        logits = self.lm_embed(x)

        return logits

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits = self(idx_cond)
            logits = logits[:, -1, :]
            idx_next = tf.random.categorical(logits, num_samples=1, seed=1337)
            idx = tf.concat([idx, idx_next], axis=1)

        return idx

In [23]:
block_size=256
batch_size=64
n_embed = 384
n_head = 6
n_layer = 6

m = BigramLanguageModel(vocab_size, block_size=block_size, n_layer=n_layer, n_head=n_head, n_embed=n_embed, dropout_rate=0.2)
logits = m(xb)
print(logits.shape)

(4, 8, 65)


In [24]:
m.compile()

In [25]:
m.summary()

Model: "bigram_language_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24960     
                                                                 
 embedding_1 (Embedding)     multiple                  98304     
                                                                 
 sequential_6 (Sequential)   (4, 8, 384)               10626048  
                                                                 
 layer_normalization_12 (Lay  multiple                 768       
 erNormalization)                                                
                                                                 
 dense_12 (Dense)            multiple                  25025     
                                                                 
Total params: 10,775,105
Trainable params: 10,775,105
Non-trainable params: 0
_________________________________

In [26]:
# steps_per_epoch=5000,
      

m.fit(get_dataset("train", block_size=block_size, batch_size=batch_size),
     validation_data=get_dataset("val", block_size=block_size, batch_size=batch_size),
     steps_per_epoch=5000,
      validation_steps=500,
      shuffle=False)

5000/5000 [==============================] - 3174s 629ms/step - loss: 1.4587 - val_loss: 1.6183


In [27]:
print(decode(m.generate(tf.zeros((1,1), dtype=tf.int64), max_new_tokens=1500)[0].numpy().tolist()))


That ever ever flow have ready were crown.

Servant:
Think you we that faults are salisk that this, by
this fly head, and that am instrumented with
honesty, or no? or who?

PETRUCHIO:
Vicested and the first served fish, which Barnardine
sufficiences have scand make joy a joyful for
you to death. Here's the way to pleasure out of it;
whom we'll bear the feast, for the content,
imputation of an end court. The other is required
More frequal diving, three eyes to choose the custom, as
please thou wh it was before thy bed, he's
kind to me; what's no present to give him? I
have done whom I must be made known that thee with for
my heart; but thou dost consent to him wish this
condition for Claudio.

Clown:
Indeed, sir.

POMPEY:
'Tis a son, a fond and elder, a virtuous worth.

DUKE VINCENTIO:
It is a Marcius in Claudio's this bed. Master
allowed with our country renient may death. What is here
matter did loss in Vienna? Plashy, madam, I were alDouble as steel;
skell do the senators of their w